In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(rc = {'figure.figsize':(15,8)})
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import random as rd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from statsmodels.tsa.vector_ar.var_model import VAR
from copy import deepcopy
from tqdm import tqdm
from matplotlib import pyplot
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
#from keras.models import Sequential
#from keras.layers import Dense
#import tensorflow as tf

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [1]:
shortlist_coutries = ['Argentina', 'Brazil', 'Canada', 
       'France', 'Germany', 'Ireland', 'Italy', 
       'Mexico', 'Spain',  
       'United Kingdom of Great Britain and Northern Ireland','United States of America']

# Value of Production

In [3]:
vp_beef = ['Milk, whole fresh cow', 'Meat, cattle']

In [4]:
vp = pd.read_csv("Value_of_Production.csv", encoding='latin1')
vp = vp.drop(['Area Code', 'Item Code', 'Element Code', 'Year Code', 'Flag'], axis=1)
vp = vp.loc[vp.Area.isin(shortlist_coutries)]
vp = vp.loc[vp.Item.isin(vp_beef)]
vp = vp.sort_values(by=["Area","Item", "Year"])
vp.reset_index(inplace=True,drop=True)
vp

#SLC standing for Standard Local Currency will require conversion.

,Area,Item,Element,Year,Unit,Value
0,Argentina,"Meat, cattle",Gross Production Value (constant 2014-2016 tho...,1961,1000 SLC,114892238.0
1,Argentina,"Meat, cattle",Gross Production Value (constant 2014-2016 tho...,1961,1000 US$,10525723.0
2,Argentina,"Meat, cattle",Gross Production Value (constant 2014-2016 tho...,1962,1000 SLC,127412815.0
3,Argentina,"Meat, cattle",Gross Production Value (constant 2014-2016 tho...,1962,1000 US$,11672782.0
4,Argentina,"Meat, cattle",Gross Production Value (constant 2014-2016 tho...,1963,1000 SLC,139542342.0
...,...,...,...,...,...,...
4494,United States of America,"Milk, whole fresh cow",Gross Production Value (constant 2014-2016 tho...,2019,1000 Int. $,41222526.0
4495,United States of America,"Milk, whole fresh cow",Gross Production Value (current thousand SLC),2019,1000 SLC,40684566.0
4496,United States of America,"Milk, whole fresh cow",Gross Production Value (constant 2014-2016 tho...,2019,1000 SLC,41933930.0
4497,United States of America,"Milk, whole fresh cow",Gross Production Value (current thousand US$),2019,1000 US$,40684566.0


# Trade:
### Imports by Value and Quantity
### Exports by Value and Quantity

In [5]:
trade_cl_beef = ['Meat, cattle', 'Milk, whole fresh cow']

In [6]:
trade_cl = pd.read_csv("Trade_CropsLivestock.csv", encoding='latin1')
trade_cl = trade_cl.loc[trade_cl.Year >= 1990]
trade_cl = trade_cl.drop(['Area Code', 'Item Code', 'Element Code', 'Year Code', 'Flag'], axis=1)
trade_cl = trade_cl.loc[trade_cl.Area.isin(shortlist_coutries)]
trade_cl = trade_cl.loc[trade_cl.Item.isin(trade_cl_beef)]
trade_cl = trade_cl.sort_values(by=["Area","Item", "Year"])
trade_cl.reset_index(inplace=True,drop=True)
trade_cl

,Area,Item,Element,Year,Unit,Value
0,Argentina,"Meat, cattle",Import Quantity,1990,tonnes,0.0
1,Argentina,"Meat, cattle",Import Value,1990,1000 US$,0.0
2,Argentina,"Meat, cattle",Export Quantity,1990,tonnes,16270.0
3,Argentina,"Meat, cattle",Export Value,1990,1000 US$,19583.0
4,Argentina,"Meat, cattle",Import Quantity,1991,tonnes,3418.0
...,...,...,...,...,...,...
2715,United States of America,"Milk, whole fresh cow",Export Value,2019,1000 US$,90078.0
2716,United States of America,"Milk, whole fresh cow",Import Quantity,2020,tonnes,18121.0
2717,United States of America,"Milk, whole fresh cow",Import Value,2020,1000 US$,16702.0
2718,United States of America,"Milk, whole fresh cow",Export Quantity,2020,tonnes,100829.0


In [7]:
trade_cl_quantity = trade_cl.loc[(trade_cl.Element == "Export Quantity")|(trade_cl.Element == "Import Quantity")]
trade_cl_value = trade_cl.loc[(trade_cl.Element == "Export Value")|(trade_cl.Element == "Import Value")]
trade_cl_quantity.reset_index(inplace=True,drop=True)
trade_cl_value.reset_index(inplace=True,drop=True)

In [8]:
years = range(1961,2021) #Year to analyse

In [9]:
#Creation of missing years to quantity
for c in trade_cl_quantity.Area.unique():
  country = trade_cl_quantity.loc[trade_cl_quantity.Area == c]
  country.reset_index(inplace=True,drop=True)
  for item in country.Item.unique():
    for year in years:
      country_fitered = country.loc[(country.Year == year) & (country.Item == item)]
      country_filt_import = country_fitered.loc[country_fitered.Element == "Import Quantity"]
      country_filt_import.reset_index(inplace=True,drop=True)
      if len(country_filt_import) < 1:
        series = country.loc[0]
        series["Item"] = item
        series["Element"] = "Import Quantity"
        series["Year"] = year
        series["Value"] = np.nan
        trade_cl_quantity=trade_cl_quantity.append(series)
      
      country_filt_export = country_fitered.loc[country_fitered.Element == "Export Quantity"]
      country_filt_export.reset_index(inplace=True,drop=True)
      if len(country_filt_export) < 1:
        series = country.loc[0]
        series["Item"] = item
        series["Element"] = "Export Quantity"
        series["Year"] = year
        series["Value"] = np.nan
        trade_cl_quantity=trade_cl_quantity.append(series)

In [10]:
for c in trade_cl_quantity.Area.unique():
  total = trade_cl_quantity.loc[trade_cl_quantity.Area == c].shape[0]
  n = trade_cl_quantity.loc[(trade_cl_quantity.Value.isnull()) & (trade_cl_quantity.Area == c)]
  nulls = n.shape[0]
  print("Percentage total of null for country {0} is {1}".format(c,round((nulls/total)*100,2)))
  print("\nMissing items:",n.Item.value_counts())
  print("\n\n")


Percentage total of null for country Argentina is 50.0

Missing items: Meat, cattle             61
Milk, whole fresh cow    59
Name: Item, dtype: int64



Percentage total of null for country Brazil is 48.33

Missing items: Meat, cattle             58
Milk, whole fresh cow    58
Name: Item, dtype: int64



Percentage total of null for country Canada is 48.33

Missing items: Meat, cattle             58
Milk, whole fresh cow    58
Name: Item, dtype: int64



Percentage total of null for country France is 48.33

Missing items: Meat, cattle             58
Milk, whole fresh cow    58
Name: Item, dtype: int64



Percentage total of null for country Germany is 48.33

Missing items: Meat, cattle             58
Milk, whole fresh cow    58
Name: Item, dtype: int64



Percentage total of null for country Ireland is 48.33

Missing items: Meat, cattle             58
Milk, whole fresh cow    58
Name: Item, dtype: int64



Percentage total of null for country Italy is 48.33

Missing items: Meat, catt

In [11]:
#Creation of missing years to Value
for c in trade_cl_value.Area.unique():
  country = trade_cl_value.loc[trade_cl_value.Area == c] #Filter by country
  country.reset_index(inplace=True,drop=True) #Reset Index
  for item in country.Item.unique():
    for year in years:
      country_fitered = country.loc[(country.Year == year) & (country.Item == item)]
      country_filt_import = country_fitered.loc[country_fitered.Element == "Import Value"]
      country_filt_import.reset_index(inplace=True,drop=True)
      if len(country_filt_import) < 1:
        series = country.loc[0]
        series["Item"] = item
        series["Element"] = "Import Value"
        series["Year"] = year
        series["Value"] = np.nan
        trade_cl_value=trade_cl_value.append(series)
      
      country_filt_export = country_fitered.loc[country_fitered.Element == "Export Value"]
      country_filt_export.reset_index(inplace=True,drop=True)
      if len(country_filt_export) < 1:
        series = country.loc[0]
        series["Item"] = item
        series["Element"] = "Export Value"
        series["Year"] = year
        series["Value"] = np.nan
        trade_cl_value=trade_cl_value.append(series)


In [12]:
for c in trade_cl_value.Area.unique():
  total = trade_cl_value.loc[trade_cl_value.Area == c].shape[0]
  n = trade_cl_value.loc[(trade_cl_value.Value.isnull()) & (trade_cl_value.Area == c)]
  nulls = n.shape[0]
  print("Percentage total of null for country {0} is {1}".format(c,round((nulls/total)*100,2)))
  print("\nMissing items:",n.Item.value_counts())
  print("\n\n")

Percentage total of null for country Argentina is 50.0

Missing items: Meat, cattle             61
Milk, whole fresh cow    59
Name: Item, dtype: int64



Percentage total of null for country Brazil is 48.33

Missing items: Meat, cattle             58
Milk, whole fresh cow    58
Name: Item, dtype: int64



Percentage total of null for country Canada is 48.33

Missing items: Meat, cattle             58
Milk, whole fresh cow    58
Name: Item, dtype: int64



Percentage total of null for country France is 48.33

Missing items: Meat, cattle             58
Milk, whole fresh cow    58
Name: Item, dtype: int64



Percentage total of null for country Germany is 48.33

Missing items: Meat, cattle             58
Milk, whole fresh cow    58
Name: Item, dtype: int64



Percentage total of null for country Ireland is 48.33

Missing items: Meat, cattle             58
Milk, whole fresh cow    58
Name: Item, dtype: int64



Percentage total of null for country Italy is 48.33

Missing items: Meat, catt

In [13]:
trade_cl_qt_filled = pd.DataFrame(columns=trade_cl_quantity.columns)
for c in trade_cl_quantity.Area.unique():
  for item in trade_cl_quantity.Item.unique():
    for element in trade_cl_quantity.Element.unique():
      country = trade_cl_quantity.loc[(trade_cl_quantity.Area == c)&(trade_cl_quantity.Item == item)& (trade_cl_quantity.Element == element )]
      country = country.sort_values(by="Year")#.interpolate()
      mean = round(country.Value.mean()) #Fill with mean
      country = country.fillna(mean)
      trade_cl_qt_filled = trade_cl_qt_filled.append(country)
      trade_cl_qt_filled.reset_index(drop=True,inplace=True)



In [14]:
trade_cl_vl_filled = pd.DataFrame(columns=trade_cl_value.columns)
for c in trade_cl_value.Area.unique():
  for item in trade_cl_value.Item.unique():
    for element in trade_cl_value.Element.unique():
      country = trade_cl_value.loc[(trade_cl_value.Area == c)&(trade_cl_value.Item == item)& (trade_cl_value.Element == element )]
      country = country.sort_values(by="Year")#.interpolate()
      mean = round(country.Value.mean()) #Fill with mean
      country = country.fillna(mean)
      trade_cl_vl_filled = trade_cl_vl_filled.append(country)
      trade_cl_vl_filled.reset_index(drop=True,inplace=True)



In [15]:
#Value in dolars
trade_cl_vl_filled["Total_value"] = trade_cl_vl_filled.Value * 1000
trade_cl_qt_filled["Total_value"] = trade_cl_qt_filled.Value * 1000


In [32]:
trade_cl_vl_filled.to_csv(r'/Users/robertobruxel/Desktop/fao/trade_cl_vl.csv', index = False)
trade_cl_qt_filled.to_csv(r'/Users/robertobruxel/Desktop/fao/trade_cl_qt.csv', index = False)

# Land Use

In [16]:
land_use_item = ['Country area', 'Land area', 'Agriculture']

land_use = pd.read_csv('Inputs_LandUse_E_All_Data.csv', encoding='latin1')
land_use = land_use.loc[land_use.Item.isin(land_use_item)]
land_use = land_use.loc[land_use.Area.isin(shortlist_coutries)]
land_use = land_use.drop(['Area Code', 'Item Code', 'Element Code', 'Year Code', 'Flag'], axis=1)
land_use.reset_index(inplace=True,drop=True)
land_use

,Area,Item,Element,Year,Unit,Value
0,Argentina,Country area,Area,1961,1000 ha,278040.0000
1,Argentina,Country area,Area,1962,1000 ha,278040.0000
2,Argentina,Country area,Area,1963,1000 ha,278040.0000
3,Argentina,Country area,Area,1964,1000 ha,278040.0000
4,Argentina,Country area,Area,1965,1000 ha,278040.0000
...,...,...,...,...,...,...
1942,United States of America,Agriculture,Area,2015,1000 ha,404718.7000
1943,United States of America,Agriculture,Area,2016,1000 ha,405264.6000
1944,United States of America,Agriculture,Area,2017,1000 ha,405810.3538
1945,United States of America,Agriculture,Area,2018,1000 ha,405810.3538


In [17]:
land_use = pd.read_csv('Inputs_LandUse_E_All_Data.csv', encoding='latin1')
land_use = land_use.loc[land_use.Item == "Agriculture"]

# Production

In [18]:
prod_item = ['Meat, cattle', 'Milk, whole fresh cow']

prod = pd.read_csv('Production_Crops_Livestock_E_All_Data.csv', encoding='latin1')
prod = prod.loc[prod.Item.isin(prod_item)]
prod = prod.loc[prod.Area.isin(shortlist_coutries)]
prod = prod.drop(['Area Code', 'Item Code', 'Element Code', 'Year Code', 'Flag'], axis=1)
prod.reset_index(inplace=True,drop=True)
prod

,Area,Item,Element,Year,Unit,Value
0,Argentina,"Meat, cattle",Yield/Carcass Weight,1961,hg/An,2100.0
1,Argentina,"Meat, cattle",Yield/Carcass Weight,1962,hg/An,2018.0
2,Argentina,"Meat, cattle",Yield/Carcass Weight,1963,hg/An,2015.0
3,Argentina,"Meat, cattle",Yield/Carcass Weight,1964,hg/An,2156.0
4,Argentina,"Meat, cattle",Yield/Carcass Weight,1965,hg/An,2184.0
...,...,...,...,...,...,...
3955,United States of America,"Milk, whole fresh cow",Milk Animals,2016,Head,9312400.0
3956,United States of America,"Milk, whole fresh cow",Milk Animals,2017,Head,9368500.0
3957,United States of America,"Milk, whole fresh cow",Milk Animals,2018,Head,9432100.0
3958,United States of America,"Milk, whole fresh cow",Milk Animals,2019,Head,9353400.0


In [19]:
prod_processed = prod.loc[(prod.Element == "Production")&(prod.Year != 2020)]
prod_processed.Element.unique()

array(['Production'], dtype=object)

In [20]:
prod_processed

,Area,Item,Element,Year,Unit,Value
60,Argentina,"Meat, cattle",Production,1961,tonnes,2145064.0
61,Argentina,"Meat, cattle",Production,1962,tonnes,2378826.0
62,Argentina,"Meat, cattle",Production,1963,tonnes,2605287.0
63,Argentina,"Meat, cattle",Production,1964,tonnes,2019240.0
64,Argentina,"Meat, cattle",Production,1965,tonnes,1995096.0
...,...,...,...,...,...,...
3894,United States of America,"Milk, whole fresh cow",Production,2015,tonnes,94618978.0
3895,United States of America,"Milk, whole fresh cow",Production,2016,tonnes,96366267.0
3896,United States of America,"Milk, whole fresh cow",Production,2017,tonnes,97761519.0
3897,United States of America,"Milk, whole fresh cow",Production,2018,tonnes,98687302.0


# Merge Datasets

In [21]:
#Dataframe to hold the values
total_df = prod_processed.sort_values(by=["Area","Item","Year"])[["Area","Item","Year"]]
total_df

,Area,Item,Year
60,Argentina,"Meat, cattle",1961
61,Argentina,"Meat, cattle",1962
62,Argentina,"Meat, cattle",1963
63,Argentina,"Meat, cattle",1964
64,Argentina,"Meat, cattle",1965
...,...,...,...
3894,United States of America,"Milk, whole fresh cow",2015
3895,United States of America,"Milk, whole fresh cow",2016
3896,United States of America,"Milk, whole fresh cow",2017
3897,United States of America,"Milk, whole fresh cow",2018


In [22]:
#Create the dataframes to merge by columns filtering trade_cl dataframe by Element
trade_import_qt = trade_cl_qt_filled.loc[(trade_cl_qt_filled.Element == "Import Quantity")&(trade_cl_qt_filled.Year != 2020)]
trade_export_qt = trade_cl_qt_filled.loc[(trade_cl_qt_filled.Element == "Export Quantity")&(trade_cl_qt_filled.Year != 2020)]
trade_import_vl = trade_cl_vl_filled.loc[(trade_cl_vl_filled.Element == "Import Value")&(trade_cl_vl_filled.Year != 2020)]
trade_export_vl = trade_cl_vl_filled.loc[(trade_cl_vl_filled.Element == "Export Value")&(trade_cl_vl_filled.Year != 2020)]


#Merge all the Values from dataframes into just one
total_df = pd.merge(total_df,prod_processed[["Area","Item","Year","Value"]],how="left",left_on=['Area','Item',"Year"],right_on=['Area','Item',"Year"])
total_df = pd.merge(total_df,trade_import_qt[["Area","Item","Year","Value"]],how="left",left_on=['Area','Item',"Year"],right_on=['Area','Item',"Year"])
total_df = pd.merge(total_df,trade_export_qt[["Area","Item","Year","Value"]],how="left",left_on=['Area','Item',"Year"],right_on=['Area','Item',"Year"])
total_df = pd.merge(total_df,trade_import_vl[["Area","Item","Year","Value"]],how="left",left_on=['Area','Item',"Year"],right_on=['Area','Item',"Year"])
total_df = pd.merge(total_df,trade_export_vl[["Area","Item","Year","Value"]],how="left",left_on=['Area','Item',"Year"],right_on=['Area','Item',"Year"])
total_df = pd.merge(total_df,land_use[["Area","Year","Value"]],how="left",left_on=['Area',"Year"],right_on=['Area',"Year"])


#Rename Columns
total_df.columns = ["Area","Item","Year","Production_ton","Import_Qt_ton","Export_Qt_ton","Import_Vl_1000$","Export_Vl_1000$","Agri_1000ha"]
total_df

,Area,Item,Year,Production_ton,Import_Qt_ton,Export_Qt_ton,Import_Vl_1000$,Export_Vl_1000$,Agri_1000ha
0,Argentina,"Meat, cattle",1961,2145064.0,3792.0,10441.0,5150.0,17709.0,137829.0000
1,Argentina,"Meat, cattle",1962,2378826.0,3792.0,10441.0,5150.0,17709.0,136434.0000
2,Argentina,"Meat, cattle",1963,2605287.0,3792.0,10441.0,5150.0,17709.0,134875.0000
3,Argentina,"Meat, cattle",1964,2019240.0,3792.0,10441.0,5150.0,17709.0,133297.0000
4,Argentina,"Meat, cattle",1965,1995096.0,3792.0,10441.0,5150.0,17709.0,131780.0000
...,...,...,...,...,...,...,...,...,...
1293,United States of America,"Milk, whole fresh cow",2015,94618978.0,2884.0,85134.0,3032.0,72602.0,404718.7000
1294,United States of America,"Milk, whole fresh cow",2016,96366267.0,3522.0,81332.0,3626.0,69697.0,405264.6000
1295,United States of America,"Milk, whole fresh cow",2017,97761519.0,7546.0,70854.0,6945.0,62575.0,405810.3538
1296,United States of America,"Milk, whole fresh cow",2018,98687302.0,6856.0,82761.0,7172.0,68482.0,405810.3538


In [29]:
total_df.to_csv(r'/Users/robertobruxel/Desktop/fao/combined.csv', index = False)

In [23]:
ratio_df = pd.DataFrame()
#Area	Item	Year
ratio_df["Area"] = total_df["Area"] 
ratio_df["Item"] = total_df["Item"] 
ratio_df["Year"] = total_df["Year"] 

ratio_df["Production_ton"] = total_df["Production_ton"] / total_df["Agri_1000ha"]
ratio_df["Import_Qt_ton"] = total_df["Import_Qt_ton"] / total_df["Agri_1000ha"]
ratio_df["Export_Qt_ton"] = total_df["Export_Qt_ton"] / total_df["Agri_1000ha"]
ratio_df["Import_Vl_1000$"] = total_df["Import_Vl_1000$"] / total_df["Agri_1000ha"]
ratio_df["Export_Vl_1000$"] = total_df["Export_Vl_1000$"] / total_df["Agri_1000ha"]
ratio_df



,Area,Item,Year,Production_ton,Import_Qt_ton,Export_Qt_ton,Import_Vl_1000$,Export_Vl_1000$
0,Argentina,"Meat, cattle",1961,15.563227,0.027512,0.075753,0.037365,0.128485
1,Argentina,"Meat, cattle",1962,17.435727,0.027794,0.076528,0.037747,0.129799
2,Argentina,"Meat, cattle",1963,19.316308,0.028115,0.077412,0.038184,0.131299
3,Argentina,"Meat, cattle",1964,15.148428,0.028448,0.078329,0.038636,0.132854
4,Argentina,"Meat, cattle",1965,15.139596,0.028775,0.079231,0.03908,0.134383
...,...,...,...,...,...,...,...,...
1293,United States of America,"Milk, whole fresh cow",2015,233.789489,0.007126,0.210354,0.007492,0.179389
1294,United States of America,"Milk, whole fresh cow",2016,237.786046,0.008691,0.200689,0.008947,0.171979
1295,United States of America,"Milk, whole fresh cow",2017,240.904447,0.018595,0.174599,0.017114,0.154198
1296,United States of America,"Milk, whole fresh cow",2018,243.185767,0.016895,0.20394,0.017673,0.168754


In [31]:
ratio_df.to_csv(r'/Users/robertobruxel/Desktop/fao/combined_ratio.csv', index = False)